# ESG Scores Data Cleaning

## 1. Data inspection

In [1]:
# import libraries
import pandas as pd
import re

In [2]:
# load data
df = pd.read_csv('ESG Scores.txt', sep='\t')

In [3]:
# show first 20 rows
df.head(20)

,RIC,Year,ESG Score,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score
0,1COV.DE,12/31/2015,52.44994215,58.93181287,47.27886179,49.760746
1,1COV.DE,12/31/2016,66.40156532,70.25422296,68.00431533,56.879433
2,1COV.DE,12/31/2017,68.92980708,68.5188051,74.68608474,59.902218
3,1COV.DE,12/31/2018,73.81273346,72.90604674,74.88865943,73.591450
4,1COV.DE,12/31/2019,74.88870359,69.74853631,72.42356247,88.146710
5,1COV.DE,12/31/2020,76.32960333,74.07848625,70.39162615,90.368149
6,1COV.DE,12/31/2021,75.82485143,75.30081329,72.189525,82.910511
7,1COV.,12/31/2022,81.24769221,82.50352674,73.7288222,91.769359
8,1COV.DE,12/31/2023,71.4575025,63.99739649,70.488781,86.278468
9,1U1.DE,12/31/2017,24.55020667,46.34944526,24.33128266,8.276521


Here, we can already see a few errors:
- 1COV. in the RIC column
- .DE in the RIC column
- Tuesday, December 31, 2019 in the Year column
- the column is called Year, but contains full dates. But the month and day seem to be always the same?

In [4]:
# check number of rows and columns
df.shape

(973, 6)

In [5]:
# check data types and missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RIC                         972 non-null    object 
 1   Year                        970 non-null    object 
 2   ESG Score                   970 non-null    object 
 3   Environmental Pillar Score  970 non-null    object 
 4   Social Pillar Score         970 non-null    object 
 5   Governance Pillar Score     970 non-null    float64
dtypes: float64(1), object(5)
memory usage: 45.7+ KB


More errors spotted:
- There seem to be missing values.
- The data types of columns 1-4 are incorrect.

In [6]:
# show summary statistics to find outliers/errors
df.describe()

,Governance Pillar Score
count,970.000000
mean,60.456328
std,21.300828
min,4.739667
25%,44.872180
50%,64.133126
75%,77.946685
max,97.324671


Have to check the summary statistics later again after fixing the data types.

## 2. Dealing with missing values

In [7]:
# show missing values per column
df.isna().sum()

RIC                           1
Year                          3
ESG Score                     3
Environmental Pillar Score    3
Social Pillar Score           3
Governance Pillar Score       3
dtype: int64

In [8]:
# show the columns with missing values in the year column
df.loc[df['Year'].isna()]

,RIC,Year,ESG Score,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score
80,AOFG.DE,NaN,NaN,NaN,NaN,NaN
281,EKTG.DE,NaN,NaN,NaN,NaN,NaN
685,P911_p.DE,NaN,NaN,NaN,NaN,NaN


We seem to have 3 rows with missing values for all columns (except RIC).
And 1 row with a missing value in the RIC column.

Since we do not know which values there should be in the 3 rows, we drop these. We can deal with the missing value in RIC column later when looking at that column specifically. Maybe there are hints what should be in the column, so that we can put in the correct value and do not have to drop the whole row.

In [9]:
# drop rows with missing values in the Year column
df.dropna(subset='Year', inplace=True)

# check if missing values are gone
print(df.isna().sum())
df.loc[df['Year'].isna()]

RIC                           1
Year                          0
ESG Score                     0
Environmental Pillar Score    0
Social Pillar Score           0
Governance Pillar Score       0
dtype: int64


,RIC,Year,ESG Score,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score


## 3. Fixing errors in the RIC column

In [10]:
# show row with missing values in the RIC column
df.loc[df['RIC'].isna()]

,RIC,Year,ESG Score,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score
38,NaN,12/31/2010,66.32451998,83.96977076,81.52632527,32.777778


This does not give us enough information to be able to infer which value we should put in. Let's check how the rows before and after look like. The data seemed to be ordered.

In [11]:
# show 5 rows before and after the missing value
na_index = df[df['RIC'].isna()].index[0]
df.iloc[na_index-5:na_index+5]

,RIC,Year,ESG Score,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score
33,AFXG.DE,9/30/2018,61.47228047,42.95288754,82.9638712,41.817403
34,AFXG.DE,9/30/2019,63.40023663,67.52013141,81.77682655,37.281776
35,AFXG.DE,9/30/2020,69.20154509,68.41253377,82.00333553,52.714060
36,AFXG.DE,9/30/2021,65.85256005,69.52197802,80.80181851,44.456975
37,AFXG.DE,9/30/2022,68.37350256,67.91516994,75.13924779,59.679160
38,NaN,12/31/2010,66.32451998,83.96977076,81.52632527,32.777778
39,AIRG.DE,12/31/2011,66.57175916,93.03102324,76.75982239,33.357813
40,AIRG.DE,12/31/2012,74.98712188,96.42432731,84.87271758,45.866030
41,AIRG.DE,12/31/2013,75.3105932,94.42651669,82.0255872,52.189702
42,AIRG.DE,12/31/2014,72.68464007,95.30162386,76.74930028,50.588979


From this pattern, I would infer that the missing value is AIRG.DE.

In [12]:
# replace missing value with correct value
df.loc[df['RIC'].isna(), 'RIC'] = 'AIRG.DE'

In the initial inspection we already saw that the RIC column contained some strange values. Let's check this by looking at values that appear less often in this column.

In [13]:
# get unique RIC values
ric_counts = df['RIC'].value_counts()

# look at values that appear less often (change the number from higher to lower values to narrow it down)
ric_counts[ric_counts < 2]

RIC
('AT1.DE)       1
1COV.           1
ALVG.Deutsch    1
.DE             1
Name: count, dtype: int64

Next, we try to find the values that are actually meant.

In [14]:
# look at rows with similar values as the incorrect values
print(df[df['RIC']=='AT1.DE'])
print(df[df['RIC'].str.startswith("1COV")])
print(df[df['RIC']=='ALVG.DE'])

       RIC        Year    ESG Score Environmental Pillar Score  \
81  AT1.DE  12/31/2017  57.97118122                32.31898802   
82  AT1.DE  12/31/2018  59.55071134                45.75734537   
83  AT1.DE  12/31/2019  63.97679468                55.63526685   
84  AT1.DE  12/31/2020  64.25964412                41.24617648   
86  AT1.DE  12/31/2022  64.21054483                50.93299766   

   Social Pillar Score  Governance Pillar Score  
81         81.87009069                56.479501  
82         79.03427269                51.452381  
83         83.46900014                50.598846  
84         89.63244701                58.595913  
86         86.50521716                52.521368  
       RIC        Year    ESG Score Environmental Pillar Score  \
0  1COV.DE  12/31/2015  52.44994215                58.93181287   
1  1COV.DE  12/31/2016  66.40156532                70.25422296   
2  1COV.DE  12/31/2017  68.92980708                 68.5188051   
3  1COV.DE  12/31/2018  73.81273346    

For the value '.DE' we can look at the surrounding values again.

In [15]:
# show 5 rows before and after the .DE value
de_index = df[df['RIC']=='.DE'].index[0]
df.iloc[de_index-5:de_index+5]

,RIC,Year,ESG Score,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score
7,1COV.,12/31/2022,81.24769221,82.50352674,73.7288222,91.769359
8,1COV.DE,12/31/2023,71.4575025,63.99739649,70.488781,86.278468
9,1U1.DE,12/31/2017,24.55020667,46.34944526,24.33128266,8.276521
10,1U1.DE,12/31/2018,39.09142473,49.20545841,39.53502319,30.447671
11,1U1.DE,12/31/2019,34.88469637,39.25456901,37.76029209,25.767294
12,.DE,12/31/2020,42.2837683,45.01251043,46.55546662,31.624604
13,1U1.DE,12/31/2021,57.49020034,43.29616639,69.81604552,43.652126
14,1U1.DE,12/31/2022,68.1036515,49.75833055,74.67773081,68.983716
15,8TRA.DE,"Tuesday, December 31, 2019",25.62498945,31.99037548,26.39936394,16.971235
16,8TRA.DE,12/31/2020,30.28771511,29.52954813,25.30222295,37.734050


From this information we can infer the correct RIC values and replace them.

In [16]:
# replace incorrect RIC values with correct ones
df.loc[df['RIC']=="('AT1.DE)", 'RIC'] = 'AT1.DE'
df.loc[df['RIC']=='1COV.', 'RIC'] = '1COV.DE'
df.loc[df['RIC']=='ALVG.Deutsch', 'RIC'] = 'ALVG.DE'
df.loc[df['RIC']=='.DE', 'RIC'] = '1U1.DE'

Let's check if there are more errors.

In [17]:
# check if the replacement worked and if there are more errors
ric_counts = df['RIC'].value_counts()
ric_counts[ric_counts < 3]

RIC
DTGGe.DE    2
Name: count, dtype: int64

In [18]:
# show 5 rows before and after the value
de_index = df[df['RIC']=='DTGGe.DE'].index[0]
df.iloc[de_index-5:de_index+5]

,RIC,Year,ESG Score,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score
269,DTEGn.DE,12/31/2020,86.40637203,80.41380706,94.18810717,75.393970
270,DTEGn.DE,12/31/2021,84.56937731,83.81274769,91.30728327,71.636215
271,DTEGn.DE,12/31/2022,82.77151583,82.24100226,90.87647347,66.923300
272,DTEGn.DE,12/31/2023,79.55443714,84.10646055,89.29676892,56.525765
273,DTGGe.DE,12/31/2021,85.11509017,82.89792392,89.37767852,82.186963
274,DTGGe.DE,12/31/2022,83.5577876,87.26520049,89.67037704,71.094608
275,ECVG.DE,12/31/2018,9.643416983,8.677383997,4.163194961,18.409962
276,ECVG.DE,12/31/2019,9.488368669,13.67580385,3.827181314,9.729272
277,ECVG.DE,12/31/2020,29.75335234,27.63306283,36.00626954,25.229052
278,ECVG.DE,12/31/2021,38.6647489,35.83125959,46.3440744,33.498558


This looks correct.

## 4. Fixing errors in the Year column

During the initial inspection we already saw that there was one value in the Year column that was formatted differently than the others. First, we want to find out if this is a single case or if there are more values with different formatting.

In [19]:
# use re pattern to match the formatting of the values in the Year column
pattern = r"^(0?[1-9]|1[0-2])/([0-2][0-9]|3[01])/([0-9]{4})$"

# filter for values that differ from the commmon pattern
df[~df['Year'].str.match(pattern)]

,RIC,Year,ESG Score,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score
15,8TRA.DE,"Tuesday, December 31, 2019",25.62498945,31.99037548,26.39936394,16.971235


Only one value differs from the common format in the Year column.

Furthermore, we saw in the inital inspection that the month and day seemed to be the same for all values. We can check if this assumption is true by looking at the rows which do not start with said month and day.

In [20]:
# filter rows that do not start with 12/31/
df[~df['Year'].str.startswith("12/31/")]

,RIC,Year,ESG Score,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score
15,8TRA.DE,"Tuesday, December 31, 2019",25.62498945,31.99037548,26.39936394,16.971235
32,AFXG.DE,9/30/2017,55.52628484,31.77455357,80.33742624,33.942423
33,AFXG.DE,9/30/2018,61.47228047,42.95288754,82.9638712,41.817403
34,AFXG.DE,9/30/2019,63.40023663,67.52013141,81.77682655,37.281776
35,AFXG.DE,9/30/2020,69.20154509,68.41253377,82.00333553,52.714060
...,...,...,...,...,...,...
905,TUI1n.DE,9/30/2019,84.14091648,83.49178153,86.31108076,81.469892
906,TUI1n.DE,9/30/2020,79.51499064,71.88013055,83.52491318,80.371481
907,TUI1n.DE,9/30/2021,77.55197329,69.13600996,78.79412965,83.263106
908,TUI1n.DE,9/30/2022,75.82172845,56.71372017,84.44312987,80.086834


Apparently, not all rows have the same month and day. Therefore, we should find out which other values there are and how often they appear.

In [21]:
# extract month and day only (MM/DD) by slicing the first 5 characters
df['Month_Day'] = df['Year'].str.slice(stop=5)

# get unique Month/Day values and their amount
df['Month_Day'].value_counts()

Month_Day
12/31    863
9/30/     88
11/30     14
5/31/      4
Tuesd      1
Name: count, dtype: int64

There seem to be just 4 different month/day values and most of the values are the 31st December. We conclude that this is no valuable information (together with the fact that the column is named 'Year'). Therefore, we just extract and keep the year. Then, we can also convert the datatype of the Year column to integer.

In [22]:
# keep only the year and convert it to int
df['Year'] = df['Year'].str[-4:].astype(int)

# drop Month/Day column
df.drop(columns='Month_Day', inplace=True)

# check if it worked
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 970 entries, 0 to 972
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RIC                         970 non-null    object 
 1   Year                        970 non-null    int64  
 2   ESG Score                   970 non-null    object 
 3   Environmental Pillar Score  970 non-null    object 
 4   Social Pillar Score         970 non-null    object 
 5   Governance Pillar Score     970 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 53.0+ KB
None


,RIC,Year,ESG Score,Environmental Pillar Score,Social Pillar Score,Governance Pillar Score
0,1COV.DE,2015,52.44994215,58.93181287,47.27886179,49.760746
1,1COV.DE,2016,66.40156532,70.25422296,68.00431533,56.879433
2,1COV.DE,2017,68.92980708,68.5188051,74.68608474,59.902218
3,1COV.DE,2018,73.81273346,72.90604674,74.88865943,73.591450
4,1COV.DE,2019,74.88870359,69.74853631,72.42356247,88.146710


It seems like this worked and the Year column has the data type integer now.

For the weirdly formatted value 'Tuesday, December 31, 2019' the code above should also have correctly extracted the year. We can check if it is indeed correct by looking at the respective row.

In [23]:
# show row with formerly weirdly formatted value
df.iloc[15,]

RIC                               8TRA.DE
Year                                 2019
ESG Score                     25.62498945
Environmental Pillar Score    31.99037548
Social Pillar Score           26.39936394
Governance Pillar Score         16.971235
Name: 15, dtype: object

The weird value is formatted correctly, too.

## 5. Check for duplicates

Another error could be duplicated rows.

In [24]:
# check for duplicates
df.duplicated().sum()

np.int64(0)

There do not seem to be any rows that are exactly the same.

To see if we lost important information by dropping the month and day, we can check if there are duplicates that have the same RIC and Year and maybe differed only in the month/day.

In [25]:
# check for duplicates in the RIC & Year column
df.duplicated(subset=['RIC', 'Year']).sum()

np.int64(0)

This does not seem to be the case, so we are all good.

## 6. Clean ESG Score column

In [26]:
print(df.dtypes)

RIC                            object
Year                            int64
ESG Score                      object
Environmental Pillar Score     object
Social Pillar Score            object
Governance Pillar Score       float64
dtype: object


we see three out of four Score columns seem to be corrupted